In [1]:
import pandas as pd

xls = pd.ExcelFile('/Users/kitaeklee/Desktop/ListofAccounts__EN.xlsx')
xl_file = pd.read_excel(xls)



In [2]:
# remove the first row which is empty
xl_file.drop([0], axis=0, inplace=True)

In [4]:
#Reset the index
xl_file.reset_index(inplace=True, drop=True)

In [6]:
# Apply the first row as a Header
header = xl_file.iloc[0]
xl_file.columns = header

In [13]:
# Drop the first row after apply the first row as a header
xl_file.drop([0], axis=0, inplace=True)

In [15]:
# Reset the index again
xl_file.reset_index(drop=True, inplace=True)

In [18]:
# create a new column 'address' to combine 'Company Name' and 'Country'
xl_file['address'] = xl_file['Account Name'] + ', ' + xl_file['Country']

In [19]:
# convert the data type to string
xl_file['address'] = xl_file['address'].astype(str)

In [21]:
# Create empty columns for latitudes and longitudes
xl_file['latitude'] = ''
xl_file['longitude'] = ''

In [37]:
xl_file['Full_Address'] = ''

In [22]:
xl_file

,Status,Account ID,Account Name,Country,Tier Class,Industry,Main Contact,State,Postal Code,address,latitude,longitude
0,Active,1000495,"3D Industrial Imaging Co., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"3D Industrial Imaging Co., Ltd., South Korea",,
1,Active,1000496,3H SYSTEM,South Korea,-,NaN,NaN,NaN,NaN,"3H SYSTEM, South Korea",,
2,Active,1000497,"3SSOFT CO.,LTD.",South Korea,-,NaN,NaN,NaN,NaN,"3SSOFT CO.,LTD., South Korea",,
3,Active,1000498,3ware (주)쓰리웨어,South Korea,-,NaN,NaN,NaN,NaN,"3ware (주)쓰리웨어, South Korea",,
4,Active,1000499,A&G A&G테크놀로지,South Korea,-,NaN,NaN,NaN,NaN,"A&G A&G테크놀로지, South Korea",,
...,...,...,...,...,...,...,...,...,...,...,...,...
971,Active,1000465,ZENITH&COMPANY (주)제니스 앤 컴퍼니,South Korea,-,NaN,NaN,NaN,NaN,"ZENITH&COMPANY (주)제니스 앤 컴퍼니, South Korea",,
972,Active,1000187,"ZensM CO., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"ZensM CO., Ltd., South Korea",,
973,Active,1000122,Zing Semiconductor Corporation 上海新昇半导体科技有限公司,China,-,Semiconductor,JiangLan,NaN,NaN,"Zing Semiconductor Corporation 上海新昇半导体科技有限公司, ...",,
974,Active,10000193,ZPMC SMART SOLUTIONS GROUP,China,-,NaN,"Wang, Zhenguo",NaN,NaN,"ZPMC SMART SOLUTIONS GROUP, China",,


In [25]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Def for extracting longitudes and latitudes from Google Geocoding API



import requests

GOOGLE_API_KEY = 'API_KEY' 

def extract_lat_long_via_address(address_or_zipcode):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={api_key}"
   
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
       
        '''
         The code below is useful when the postal code exists.
        '''
        #for i in results['address_components']:
            #if i['types'][0] == 'postal_code':
                 #post_code = i['long_name']    


    except:
        pass
    return lat, lng

In [28]:
# Loop through the 'address' and use Google Geocoding API to get lat and lng
# then put them in 'latitude' and 'longitude' columns.
# It takes approximately 1 second to get one pair of latitude and longitude

z=0
for address in xl_file['address']:
    if z == 977:
        break
        
    dic = extract_lat_long_via_address(address)
    
    xl_file.loc[z, ['latitude']] = dic[0]
    xl_file.loc[z, ['longitude']] = dic[1]

    
    z+=1
    

In [31]:
xl_file

,Status,Account ID,Account Name,Country,Tier Class,Industry,Main Contact,State,Postal Code,address,latitude,longitude
0,Active,1000495,"3D Industrial Imaging Co., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"3D Industrial Imaging Co., Ltd., South Korea",None,None
1,Active,1000496,3H SYSTEM,South Korea,-,NaN,NaN,NaN,NaN,"3H SYSTEM, South Korea",None,None
2,Active,1000497,"3SSOFT CO.,LTD.",South Korea,-,NaN,NaN,NaN,NaN,"3SSOFT CO.,LTD., South Korea",37.4754,127.042
3,Active,1000498,3ware (주)쓰리웨어,South Korea,-,NaN,NaN,NaN,NaN,"3ware (주)쓰리웨어, South Korea",37.4813,126.884
4,Active,1000499,A&G A&G테크놀로지,South Korea,-,NaN,NaN,NaN,NaN,"A&G A&G테크놀로지, South Korea",37.0902,-95.7129
...,...,...,...,...,...,...,...,...,...,...,...,...
971,Active,1000465,ZENITH&COMPANY (주)제니스 앤 컴퍼니,South Korea,-,NaN,NaN,NaN,NaN,"ZENITH&COMPANY (주)제니스 앤 컴퍼니, South Korea",37.1779,-93.222
972,Active,1000187,"ZensM CO., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"ZensM CO., Ltd., South Korea",None,None
973,Active,1000122,Zing Semiconductor Corporation 上海新昇半导体科技有限公司,China,-,Semiconductor,JiangLan,NaN,NaN,"Zing Semiconductor Corporation 上海新昇半导体科技有限公司, ...",31.2304,121.474
974,Active,10000193,ZPMC SMART SOLUTIONS GROUP,China,-,NaN,"Wang, Zhenguo",NaN,NaN,"ZPMC SMART SOLUTIONS GROUP, China",None,None


In [33]:
# 237 companies out of 976 did not get lat and lng returned
xl_file['latitude'].isnull().sum()

237

In [34]:
# Below is code for getting postal codes from Google Geocoding API
# But it does not return postal codes for all addresses.


#for i in dic1['address_components']:
#      if i['types'][0] == 'postal_code':
#        post_code = i['long_name']
#post_code

In [60]:
# For example, this will return the information like below
import geocoder
latlng = [xl_file.loc[4, ['latitude']]['latitude'], xl_file.loc[4, ['longitude']]['longitude']]
g = geocoder.mapbox(latlng, method='reverse', key='pk.eyJ1IjoieG9yOTIzNyIsImEiOiJja2hsYTRydHAxa3l4MnJucXB0YXZzeXFyIn0._r9Zw8FZ4rvarTcrcCJf-w')
g.json


{'address': 'Tires Tires Tire- Hamilton, 329 W 19 St, Coffeyville, Kansas 67337, United States',
 'city': 'Coffeyville',
 'country': 'United States',
 'lat': 37.09024,
 'lng': -95.712891,
 'ok': True,
 'postal': '67337',
 'quality': 1,
 'raw': {'id': 'poi.566935763453',
  'type': 'Feature',
  'place_type': ['poi'],
  'relevance': 1,
  'properties': {'foursquare': '5b815892603d2a003954f25b',
   'landmark': True,
   'address': '329 W 19 St',
   'category': 'auto repair, car repair, body shop, shop'},
  'text': 'Tires Tires Tire- Hamilton',
  'place_name': 'Tires Tires Tire- Hamilton, 329 W 19 St, Coffeyville, Kansas 67337, United States',
  'center': [-95.712891, 37.09024],
  'geometry': {'coordinates': [-95.712891, 37.09024], 'type': 'Point'},
  'context': [{'id': 'postcode.11049672724931430', 'text': '67337'},
   {'id': 'place.10598606915801060',
    'wikidata': 'Q996804',
    'text': 'Coffeyville'},
   {'id': 'region.10336703176034420',
    'wikidata': 'Q1558',
    'short_code': 'US-K

In [56]:
import geocoder

#key = token from Mapbox
z=0
for lat, lng in zip(xl_file['latitude'], xl_file['longitude']):
    latlng = [lat, lng]
    if latlng[0] == None:
        z+=1
        continue
    else:
        g = geocoder.mapbox(latlng, method='reverse', key='TOKEN')
        
        if 'postal' in g.json:
            xl_file.loc[z, ['Postal Code']] = g.json['postal']
            if 'state' in g.json:
                xl_file.loc[z,['State']] = g.json['state']
                z+=1
                continue
            elif 'city' in g.json:
                xl_file.loc[z,['State']] = g.json['city']
                z+=1
                continue
            else:
                z+=1
                continue
           
        elif 'state' in g.json:
            xl_file.loc[z,['State']] = g.json['state']
            z+=1
            continue
        elif 'city' in g.json:
            xl_file.loc[z,['State']] = g.json['city']
            z+=1
            continue
        else:
            z+=1
            continue
        




In [57]:
xl_file

,Status,Account ID,Account Name,Country,Tier Class,Industry,Main Contact,State,Postal Code,address,latitude,longitude,Full_Address
0,Active,1000495,"3D Industrial Imaging Co., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"3D Industrial Imaging Co., Ltd., South Korea",None,None,
1,Active,1000496,3H SYSTEM,South Korea,-,NaN,NaN,NaN,NaN,"3H SYSTEM, South Korea",None,None,
2,Active,1000497,"3SSOFT CO.,LTD.",South Korea,-,NaN,NaN,Seoul,06776,"3SSOFT CO.,LTD., South Korea",37.4754,127.042,"현대자동차 현대강남서비스, Seocho-gu, Seoul 06776, South K..."
3,Active,1000498,3ware (주)쓰리웨어,South Korea,-,NaN,NaN,Seoul,08510,"3ware (주)쓰리웨어, South Korea",37.4813,126.884,"부자 아줌마, 금천구 벚꽃로 298, Geumcheon-gu, Seoul 08510..."
4,Active,1000499,A&G A&G테크놀로지,South Korea,-,NaN,NaN,Kansas,67337,"A&G A&G테크놀로지, South Korea",37.0902,-95.7129,"Tires Tires Tire- Hamilton, 329 W 19 St, Coffe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,Active,1000465,ZENITH&COMPANY (주)제니스 앤 컴퍼니,South Korea,-,NaN,NaN,Missouri,65809,"ZENITH&COMPANY (주)제니스 앤 컴퍼니, South Korea",37.1779,-93.222,"Sturhahn Breazeale & Co, 3534 E Sunshine St, S..."
972,Active,1000187,"ZensM CO., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"ZensM CO., Ltd., South Korea",None,None,
973,Active,1000122,Zing Semiconductor Corporation 上海新昇半导体科技有限公司,China,-,Semiconductor,JiangLan,Shanghai Shi,NaN,"Zing Semiconductor Corporation 上海新昇半导体科技有限公司, ...",31.2304,121.474,"周生生, 龍之夢, Shanghai Shi China"
974,Active,10000193,ZPMC SMART SOLUTIONS GROUP,China,-,NaN,"Wang, Zhenguo",NaN,NaN,"ZPMC SMART SOLUTIONS GROUP, China",None,None,


In [58]:
xl_file['State'].isnull().sum()

206

In [66]:
xl_file['Postal Code'].isnull().sum()

409

In [65]:
xl_file

,Status,Account ID,Account Name,Country,Tier Class,Industry,Main Contact,State,Postal Code,address,latitude,longitude
0,Active,1000495,"3D Industrial Imaging Co., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"3D Industrial Imaging Co., Ltd., South Korea",None,None
1,Active,1000496,3H SYSTEM,South Korea,-,NaN,NaN,NaN,NaN,"3H SYSTEM, South Korea",None,None
2,Active,1000497,"3SSOFT CO.,LTD.",South Korea,-,NaN,NaN,Seoul,06776,"3SSOFT CO.,LTD., South Korea",37.4754,127.042
3,Active,1000498,3ware (주)쓰리웨어,South Korea,-,NaN,NaN,Seoul,08510,"3ware (주)쓰리웨어, South Korea",37.4813,126.884
4,Active,1000499,A&G A&G테크놀로지,South Korea,-,NaN,NaN,Kansas,67337,"A&G A&G테크놀로지, South Korea",37.0902,-95.7129
...,...,...,...,...,...,...,...,...,...,...,...,...
971,Active,1000465,ZENITH&COMPANY (주)제니스 앤 컴퍼니,South Korea,-,NaN,NaN,Missouri,65809,"ZENITH&COMPANY (주)제니스 앤 컴퍼니, South Korea",37.1779,-93.222
972,Active,1000187,"ZensM CO., Ltd.",South Korea,-,NaN,NaN,NaN,NaN,"ZensM CO., Ltd., South Korea",None,None
973,Active,1000122,Zing Semiconductor Corporation 上海新昇半导体科技有限公司,China,-,Semiconductor,JiangLan,Shanghai Shi,NaN,"Zing Semiconductor Corporation 上海新昇半导体科技有限公司, ...",31.2304,121.474
974,Active,10000193,ZPMC SMART SOLUTIONS GROUP,China,-,NaN,"Wang, Zhenguo",NaN,NaN,"ZPMC SMART SOLUTIONS GROUP, China",None,None


In [68]:
xl_file.to_excel(r'/Users/kitaeklee/Desktop/ListofAccounts__EN1.xlsx', index = False)


